# Unrolling Cylindrical Cell “Jelly Rolls” and Detecting Electrode Buckling from XCT

This notebook demonstrates a reproducible workflow to **unroll** a binary segmentation of a spiral (the jelly roll in a cylindrical Li-ion cell) from Euclidean coordinates into **polar coordinates** \((r, \theta)\). Inspired by Kok *et al.* (2019), we then extract individual layers and quantify their deviation from an **ideal Archimedean spiral**, providing a simple measure of **winding quality**.

**We will:**
1. Quickly visualize example images and masks.
2. Explain the unrolling method step by step (center, radial map, angular map, masking, unrolling).
3. Use helper functions to analyze the data (layer extraction, line fits, deviations).
4. Summarize findings.

> **Notes**: All computations below operate in pixel units for clarity. You can convert to physical units by multiplying by the pixel size of your dataset.

## 0) Setup & imports
Configure paths so `utils/` can be imported, then load the scientific stack and unrolling helpers.

In [ ]:
import os, sys
CWD = os.getcwd()
if os.path.basename(CWD) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(CWD, ".."))
else:
    PROJECT_ROOT = CWD  # fallback if already at root
os.chdir(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))
print(f"Correct Working Directory: {str(os.path.basename(os.getcwd()))=='battery_xct_workflows'}")

In [ ]:
# If you're running this notebook in Google Colab, install requirements:
import sys
if "google.colab" in sys.modules:
    # Install requirements from GitHub
    !pip install -q -r https://raw.githubusercontent.com/MPJ-Imaging/battery_xct_workflows/HEAD/requirements.txt

    # Enable ipywidgets + widget backend
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
        %matplotlib widget
        print("Colab: using Matplotlib widget backend.")
    except Exception as e:
        print("Colab: could not enable widget backend, falling back to inline.", e)
        %matplotlib inline
else:
    # Local / Binder users can uncomment if packages are missing
    # !pip install -r requirements.txt
    %matplotlib widget

In [ ]:
import tifffile as tiff
import numpy as np
import numpy.ma as ma
import skimage
from skimage.measure import label as label
from scipy.ndimage import center_of_mass as c_of_m
from scipy.ndimage import distance_transform_edt as dist_trans
import matplotlib.pyplot as plt
import math
import utils.unrolling as nrol

## 1) Visualize sample data
Load two example slices and their binary masks. *Image 1* is nominal (no buckling); *Image 2* shows buckled layers near the center. We’ll overlay masks to confirm the segmentation quality.

In [ ]:
im1 = tiff.imread('data/im_1.tif')
mask1 =tiff.imread('data/mask_1.tif')
im2 = tiff.imread('data/im_2.tif')
mask2 =tiff.imread('data/mask_2.tif')

def show_overlay(img, msk, ax, title=""):
    # basic checks
    if img.shape != msk.shape:
        raise ValueError(f"Image/mask shapes differ: {img.shape} vs {msk.shape}")

    ax.imshow(img, cmap="gray", interpolation="nearest")
    alpha = (msk > 0).astype(float) * 0.5   # 0 where mask==0, 0.5 where mask>0
    ax.imshow(msk, cmap="Reds", alpha=alpha, interpolation="nearest")
    ax.set_title(title)
    ax.axis("off")

# ---- Visualize ----
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()

# Img Msk Pair 1: image + overlay
axes[0].imshow(im1, cmap="gray", interpolation="nearest")
axes[0].set_title("Image 1")
axes[0].axis("off")
show_overlay(im1, mask1, axes[1], "Image 1 + Mask")

# Img Msk Pair 2: image + overlay
axes[2].imshow(im2, cmap="gray", interpolation="nearest")
axes[2].set_title("Image 2")
axes[2].axis("off")
show_overlay(im2, mask2, axes[3], "Image 2 + Mask")

plt.tight_layout()
plt.show()

*Observation*: In **Image 1**, the mask follows the electrode layers closely. In **Image 2**, central buckling is visible, and the mask captures it well—ideal for testing the unrolling analysis.

## 2) How the unrolling works
The core idea is to map each segmented electrode pixel from \((x, y)\) to **polar coordinates** \((r, \theta)\) around the cell center:

- **Center**: choose a reference point (here, the center of mass of the mask).
- **Radial map**: distance of each pixel to the center ⇒ defines \(r\).
- **Angular map**: angle of each pixel around the center ⇒ defines \(\theta\).

Once we have \(r\) and \(\theta\) for electrode pixels, plotting **\(r\) vs. \(\theta\)** effectively *unrolls* the spiral. In this space, an **ideal Archimedean spiral** \(r = a + b\theta\) appears as a **straight line**.

### 2.1 Choose the center
We use the **center of mass** of the electrode mask as the spiral center. We cast to integers for array indexing convenience (subpixel accuracy isn’t critical for the demo).

In [ ]:
com = c_of_m(mask1)
com = tuple(int(x) for x in com) #converting to int to remove subpixels
print('C_of_M is '+ str(com))

### 2.2 Optional: interior (casing) mask
For visualization, we generate a simple interior mask (inside the can = 1, outside = 0) via threshold + flood fill. This step is **optional** and used just to restrict displays to the cell interior.

In [ ]:
# first a basic utility mask where the inside of the battery is =1 and outside is = 0
casing_seg = np.where(im1 > 60000, 1, 0)
flooded_cell = skimage.segmentation.flood_fill(casing_seg, com, new_value = 1)
plt.figure(figsize=(5, 5))
plt.imshow(flooded_cell.astype(np.uint8), cmap='gray')
plt.show()

### 2.3 Radial distance map \(r\)
Create an image where each pixel stores its **Euclidean distance** to the center. Multiplying by the interior mask limits visualization to the can’s interior only.

In [ ]:
dist_transform = np.ones_like(im1)
dist_transform[com[0], com[1]] = 0
dist_transform = dist_trans(dist_transform.astype(np.uint16))
plt.imshow((dist_transform*flooded_cell).astype(np.uint16), cmap='viridis')
plt.colorbar()
plt.title('Radial distance wrt centre of mass of cell')
plt.show()

Each interior pixel now carries its **radial position** (in pixels) relative to the chosen center.

### 2.4 Angular map \(\theta\)
Next we assign each pixel an **angular position** around the center. With array coordinates in image space, we compute \(\theta\) using `np.arctan2` on offset coordinates. (We use the ordering consistent with the code below.)

In [ ]:
arr = np.zeros_like(im1)
arr = np.indices(arr.shape)
print(arr[0]) #arr[0] here returns the y-coord of each point in the image 
print(arr[1]) #similarly arr[1] will return x-coord

In [ ]:
y_in = (arr[0] - com[0])
x_in = (arr[1] - com[1])
angular_arr = np.arctan2(x_in,y_in)

We convert \(\theta\) to degrees in \([0, 360)\) for easier plotting and mask it to the interior for a clean view.

In [ ]:
angular_arr = angular_arr * (180/math.pi) + 180 #convert into degrees with range between 0 - 360
plt.imshow((angular_arr*flooded_cell).astype(np.uint16), cmap='viridis')
plt.colorbar()
plt.title('Angular position WRT centre of mass of cell')
plt.show()

Each interior pixel now carries its **angular position** with respect to the selected center.

### 2.5 Apply the electrode segmentation
Mask the **radial** and **angular** maps with the electrode segmentation so we only keep pixels belonging to the electrode phase.

In [ ]:
# radial
elec_radius_arr = dist_transform * np.where(mask1>0,1,0)
elec_radius_arr_masked = ma.masked_where(elec_radius_arr == 0, elec_radius_arr)
plt.imshow(elec_radius_arr.astype(np.uint16), cmap='viridis')
plt.colorbar()
plt.title('Radial position OF ELECTRODE wrt centre of mass of cell')
plt.show()

Electrode pixels are now labeled by **radial distance**.

In [ ]:
# anglular
elec_angle_arr = angular_arr * np.where(mask1>0,1,0)
elec_angle_arr_masked = ma.masked_where(elec_angle_arr == 0, elec_angle_arr)
plt.imshow(elec_angle_arr.astype(np.uint16), cmap='viridis')
plt.colorbar()
plt.title('Angular position OF ELECTRODE wrt centre of mass of cell')
plt.show()

Electrode pixels are now labeled by **angular position** as well.

Next, we reduce these masked arrays to 1D vectors (discarding masked values) so that each pixel contributes a pair \((\theta, r)\). Concatenating these yields an **unrolled** representation of the electrode.

In [ ]:
#to 1D
elec_radius_arr_masked = elec_radius_arr_masked[elec_radius_arr_masked.mask == False].ravel()
elec_angle_arr_masked = elec_angle_arr_masked[elec_angle_arr_masked.mask == False].ravel()

### 2.6 Quick unrolled scatter
Plot \(r\) versus \(\theta\) for the segmented electrode pixels. In the **ideal** case, each layer forms a straight line (Archimedean spiral in polar space). Buckling or local mis-winding appears as **nonlinearity** or **outliers**.

In [ ]:
# now lets plot 
plt.scatter(elec_angle_arr_masked, elec_radius_arr_masked, s=5) # the s=5 is the size of the points and may need to be adjusted

# Set the axis labels
plt.xlabel('Angular Position, degrees', fontsize = 10)
plt.ylabel('Radial Position, px', fontsize = 10)

# Set the plot title
plt.title('Unrolled Electrode', fontsize = 12)

# Display the plot
plt.show()

That works - but the `utils/unrolling.py` helpers provide a cleaner API for layer extraction and error metrics.

## 3) Analyze the sample data with the unrolling helpers
We’ll use `virtual_unroll` to extract layer-wise \((\theta, r)\) summaries and `add_linear_fit_errors` to compute deviations from **straight lines** (i.e., an ideal Archimedean spiral in this space).

In [ ]:
help(nrol.virtual_unroll)

In [ ]:
unrolled1 = nrol.virtual_unroll(mask1, com)
unrolled1.head()

`virtual_unroll` returns a **DataFrame** with one row per extracted layer and columns describing layer geometry in the unrolled space. The `chunk_thkn` column records layer thickness at sampled positions (useful context for deviations).

In an **ideal cell**, layers follow an **Archimedean spiral**:

$$ r = a + b\,\theta $$

which appears as a **straight line** in \((\theta, r)\). Fitting a line to each layer and computing residuals gives a direct measure of **buckling or mis-winding**.

In [ ]:
help(nrol.add_linear_fit_errors)

In [ ]:
unrolled1 = nrol.add_linear_fit_errors(unrolled1)
unrolled1.head()

Visualize the unrolled layers and their **maximum absolute error (MaxAE)** from the line fit. For nominal data, layers should be near-linear with small MaxAE values.

In [ ]:
nrol.plot_unrolled_layers(unrolled1, title='Unrolled Image 1')

Now apply the same pipeline to **Image 2**, which exhibits central buckling. We expect larger deviations from linearity (higher MaxAE), especially for layers near the center.

In [ ]:
unrolled2 = nrol.virtual_unroll(mask2)
unrolled2 = nrol.add_linear_fit_errors(unrolled2)
nrol.plot_unrolled_layers(unrolled2, title='Unrolled Image 2')

Optionally, color each layer by its deviation to highlight where buckling is most severe.

In [ ]:
nrol.plot_unrolled_layers(unrolled2, title='Unrolled Image 2', color_by_error=True)

*Observation*: The unrolled layers in **Image 2** show clear **nonlinearity** with higher **MaxAE** values, especially near the center where buckling is present (e.g., layer 2 deviates by ~15 px in this example).

## 4) Conclusion
Unrolling electrode segmentations into polar coordinates provides an intuitive view of winding quality: ideal layers appear as straight lines, while buckling manifests as curvature and increased residuals. The approach here—center selection, radial/angle maps, mask application, and simple linear fits—offers a **transparent, reproducible** baseline that you can extend with physical scaling, robust fitting, or per-layer QC thresholds for automated flagging.